# Testing notebook

In [2]:
import geopandas as gpd
import folium
import json
from tqdm import tqdm
import re

from custom_functions.gtfs_methods import detect_direction, match_schedule_for_service_line
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [29]:
obs = pd.read_pickle("data/computed/observed_schedule.pkl")

In [30]:
obs

,date,time,route_short_name,distance_from_point,stop_id,stop_id_terminus
0,19-09-2021,10:33:18,1,1,8022,8161
1,19-09-2021,10:33:18,1,1,8732,8161
2,19-09-2021,10:33:18,1,1,8071,8731
3,19-09-2021,10:33:18,1,0,8152,8161
4,19-09-2021,10:33:18,1,1,8271,8731
...,...,...,...,...,...,...
19421878,14-09-2021,12:25:54,97,232,6109,6361
19421879,14-09-2021,12:25:54,98,0,3810,1951
19421880,14-09-2021,12:25:54,98,0,1951,2382
19421881,14-09-2021,12:25:54,98,270,3815,2382


In [33]:
obs[((obs.date == "09-09-2021") | (obs.date == "10-09-2021") ) & (obs.route_short_name == "7") & (obs.stop_id == "5706")].to_csv("data/CSVs/full_obs_hembeek.csv")

In [6]:
data = pd.read_pickle("data/computed/schedule_normalized_dist.pkl")
lines = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")

In [9]:
stops_geom = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")

2022-12-07 13:05:12,253 - fiona.ogrext - INFO - collection - : Line (215) - Failed to auto identify EPSG: 7


In [10]:
sg = stops_geom[["stop_id", "descr_fr", "descr_nl", "geometry"]].drop_duplicates(
    "stop_id"
)

In [29]:
sg.stop_id.unique().shape

(2469,)

In [26]:
pd.DataFrame(stops_geom[stops_geom.stop_id.isin(stops_geom[stops_geom.duplicated(subset="stop_id")].stop_id)]).sort_values("stop_id")

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry
3642,044t,1,1,0089,MONTGOMERY,MONTGOMERY,Montgomery,Montgomery,152832.9,169733.3,T,44,TERVUREN STATION,POINT (152832.900 169733.300)
3641,039t,2,20,0089,MONTGOMERY,MONTGOMERY,Montgomery,Montgomery,152832.9,169733.3,T,39,MONTGOMERY,POINT (152832.900 169733.300)
3602,039t,1,1,0089,MONTGOMERY,MONTGOMERY,Montgomery,Montgomery,152832.9,169733.3,T,39,BAN-EIK,POINT (152832.900 169733.300)
3675,044t,2,17,0089,MONTGOMERY,MONTGOMERY,Montgomery,Montgomery,152832.9,169733.3,T,44,MONTGOMERY,POINT (152832.900 169733.300)
3229,004t,1,3,0501,DE BROUCKERE,DE BROUCKERE,De Brouckère,De Brouckère,148882.0,171233.0,T,4,STALLE (P),POINT (148882.000 171233.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,047b,1,21,9784B,VILVOORDE STATION,VILVOORDE STATION,Vilvoorde Station,Vilvoorde Station,154442.9,179359.8,B,47,VILVOORDE STATION,POINT (154442.900 179359.800)
1964,079b,1,5,9853,HOF TER MUSSCHEN,HOF TER MUSSCHEN,Hof ter Musschen,Hof ter Musschen,155563.0,171565.0,B,79,SCHUMAN,POINT (155563.000 171565.000)
650,042b,1,8,9853,HOF TER MUSSCHEN,HOF TER MUSSCHEN,Hof ter Musschen,Hof ter Musschen,155563.0,171565.0,B,42,ROODEBEEK,POINT (155563.000 171565.000)
2032,080b,1,31,9946,HAREN,HAREN,Haren,Haren,153812.0,175858.0,B,80,HAREN,POINT (153812.000 175858.000)


In [30]:
obs = pd.read_pickle("data/computed/observed_time_ES.pkl").drop(columns=["Unnamed: 0"])

In [31]:
obs

,date,time,route_short_name,distance_from_point,stop_id__current,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned,stop_id__terminus,stop_code__terminus,stop_name__terminus,stop_desc__terminus,stop_lat__terminus,stop_lon__terminus,zone_id__terminus,stop_url__terminus,location_type__terminus,parent_station__terminus,stop_id_cleaned__terminus,date_label,time_seconds,ligne_cleaned,mode,lag_-1,lag_+1,t_lag_before,t_diff(sec),adjusted_arrival_time(ts),adjusted_arrival_time
0,06-09-2021,10:15:32,8,0,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,36932,8,T,NaN,38.0,NaN,NaN,36932.000,10:15:32
1,06-09-2021,10:16:04,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,36964,8,T,0.0,140.0,36932.0,32.0,36955.607,10:15:55
2,06-09-2021,19:15:56,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,69356,8,T,140.0,181.0,37030.0,32326.0,69347.607,19:15:47
3,06-09-2021,19:15:56,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,69356,8,T,140.0,181.0,37030.0,32326.0,69347.607,19:15:47
4,06-09-2021,19:15:56,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,69356,8,T,140.0,181.0,37030.0,32326.0,69347.607,19:15:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14272873,21-09-2021,13:02:41,27,36,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,1152,NaN,MEUDON,NaN,50.844895,4.416773,NaN,NaN,0.0,NaN,1152,workdays,46961,27,B,NaN,132.0,NaN,NaN,46952.692,13:02:32
14272874,21-09-2021,13:02:41,27,36,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,1152,NaN,MEUDON,NaN,50.844895,4.416773,NaN,NaN,0.0,NaN,1152,workdays,46961,27,B,NaN,132.0,NaN,NaN,46952.692,13:02:32
14272875,21-09-2021,13:03:13,27,132,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,1152,NaN,MEUDON,NaN,50.844895,4.416773,NaN,NaN,0.0,NaN,1152,workdays,46993,27,B,36.0,360.0,46961.0,32.0,46962.538,13:02:42
14272876,21-09-2021,00:18:14,27,58,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,2289,NaN,MONTGOMERY,NaN,50.837869,4.407055,NaN,NaN,0.0,NaN,2289,workdays,1094,27,B,NaN,307.0,NaN,NaN,1080.615,00:18:00


In [12]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                               'Parrot', 'Parrot'],
                    'Max Speed': [380., 370., 24., 26.]})

,Animal,Max Speed
0,Falcon,380.0
1,Falcon,370.0
2,Parrot,24.0
3,Parrot,26.0


In [13]:
df["used"] = np.nan
df 

,Animal,Max Speed,used
0,Falcon,380.0,NaN
1,Falcon,370.0,NaN
2,Parrot,24.0,NaN
3,Parrot,26.0,NaN


In [10]:
df_g = df.groupby("Animal")
df_g

In [22]:
val = pd.DataFrame({'player': ['hakimi', 'ronaldo',
                               'test', 'tigre'],
                    'animal' : ['Falcon', 'Falcon',
                               'Parrot', 'Parrot'],
                    'speed_player': [385., 200., 90., 55.]})
val

,player,animal,speed_player
0,hakimi,Falcon,385.0
1,ronaldo,Falcon,200.0
2,test,Parrot,90.0
3,tigre,Parrot,55.0


In [28]:

for rec in val.to_dict("records"):
    print(rec["player"])

    es = df_g.get_group(rec["animal"])
    es["diff_p"] = abs(es["Max Speed"] - rec["speed_player"])
    min_s = es[es["diff_p"] == min(es.diff_p)]

    print(min_s)



hakimi
   Animal  Max Speed  used  diff_p
0  Falcon      380.0   NaN     5.0
ronaldo
   Animal  Max Speed  used  diff_p
1  Falcon      370.0   NaN   170.0
test
   Animal  Max Speed  used  diff_p
3  Parrot       26.0   NaN    64.0
tigre
   Animal  Max Speed  used  diff_p
3  Parrot       26.0   NaN    29.0


In [2]:
stops = pd.read_csv("data/timetables/gtfs3Sept/stops.txt")
coord = stops[["stop_id", "stop_lat", "stop_lon"]]

In [4]:
coord.to_pickle("data/computed/stops.geo.pkl")

In [5]:
data = pd.read_pickle("data/computed/matches_EWT_delay.pkl")

In [6]:
data

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,delay,delay_label
0,112961156236299000,09:45:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,35100,10.0,cluster_2,10.0,1,2021-09-01,2021-09-17,06-09-2021,NaN,116.0,89,9556,TERVUREN STATION,9556,workdays,44,T,35660,09:54:20,79122,NaN,NaN,4.679775,5.845066,1.165291,560,lat
1,112960577236299000,09:54:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,35640,9.0,cluster_2,10.0,1,2021-09-01,2021-09-17,06-09-2021,NaN,116.0,89,9556,TERVUREN STATION,9556,workdays,44,T,35660,09:54:20,77342,NaN,0.000000,4.679775,5.845066,1.165291,20,nan
2,112961222236299000,09:58:00,0089,1,MONTGOMERY,40,236299000,MUSEE DU TRAM,0,44,MONTGOMERY - TERVUREN STATION,35880,4.0,cluster_2,10.0,1,2021-09-01,2021-09-17,06-09-2021,NaN,0.0,89,5281G,MUSEE DU TRAM,5281,workdays,44,T,36100,10:01:40,79162,NaN,7.333333,4.679775,5.845066,1.165291,220,lat
3,112960576236299000,10:04:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,36240,6.0,cluster_2,10.0,1,2021-09-01,2021-09-17,06-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,36259,10:04:19,77332,NaN,2.650000,4.679775,5.845066,1.165291,19,nan
4,112960587236299000,10:14:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,36840,10.0,cluster_2,10.0,1,2021-09-01,2021-09-17,06-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,36868,10:14:28,77382,NaN,10.150000,4.679775,5.845066,1.165291,28,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177329,113357527237592001,09:25:00,9996,17,MONTGOMERY,59,237592001,MONTGOMERY,0,61,GARE DU NORD - MONTGOMERY,33900,40.0,cluster_0,44.5,0,2021-09-01,2021-09-17,17-09-2021,NaN,0.0,9996,9996,MONTGOMERY,9996,workdays,61,B,36550,10:09:10,156057,NaN,NaN,27.991892,410.958333,382.966441,2650,lat
4177330,113357523237592001,10:11:00,9996,17,MONTGOMERY,59,237592001,MONTGOMERY,0,61,GARE DU NORD - MONTGOMERY,36660,46.0,cluster_0,44.5,0,2021-09-01,2021-09-17,17-09-2021,NaN,0.0,9996,9996,MONTGOMERY,9996,workdays,61,B,36550,10:09:10,156017,NaN,0.000000,27.991892,410.958333,382.966441,-110,adv
4177331,113357546237592001,23:35:00,9996,17,MONTGOMERY,59,237592001,MONTGOMERY,0,61,GARE DU NORD - MONTGOMERY,84900,79.0,cluster_0,44.5,0,2021-09-01,2021-09-17,17-09-2021,NaN,0.0,9996,9996,MONTGOMERY,9996,workdays,61,B,85865,23:51:05,156247,NaN,821.916667,27.991892,410.958333,382.966441,965,lat
4177332,113357559237592001,24:15:00,9996,17,MONTGOMERY,59,237592001,MONTGOMERY,0,61,GARE DU NORD - MONTGOMERY,87300,20.0,cluster_0,44.5,0,2021-09-01,2021-09-17,17-09-2021,NaN,0.0,9996,9996,MONTGOMERY,9996,workdays,61,B,85865,23:51:05,156377,NaN,0.000000,27.991892,410.958333,382.966441,-1435,adv


In [3]:
pd.read_pickle("data/computed/observed_time_ES.pkl")

,date,time,route_short_name,distance_from_point,stop_id__current,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_id_cleaned,stop_id__terminus,stop_code__terminus,stop_name__terminus,stop_desc__terminus,stop_lat__terminus,stop_lon__terminus,zone_id__terminus,stop_url__terminus,location_type__terminus,parent_station__terminus,stop_id_cleaned__terminus,date_label,time_seconds,ligne_cleaned,mode,lag_-1,lag_+1,t_lag_before,t_diff(sec),adjusted_arrival_time(ts),adjusted_arrival_time
0,06-09-2021,10:15:32,8,0,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,36932,8,T,NaN,38.0,NaN,NaN,36932.000,10:15:32
1,06-09-2021,10:16:04,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,36964,8,T,0.0,140.0,36932.0,32.0,36955.607,10:15:55
2,06-09-2021,19:15:56,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,69356,8,T,140.0,181.0,37030.0,32326.0,69347.607,19:15:47
3,06-09-2021,19:15:56,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,69356,8,T,140.0,181.0,37030.0,32326.0,69347.607,19:15:47
4,06-09-2021,19:15:56,8,38,5409,NaN,ABBAYE,NaN,50.818926,4.370739,NaN,NaN,0.0,NaN,5409,2351,NaN,BUYL,NaN,50.818914,4.378791,NaN,NaN,0.0,NaN,2351,workdays,69356,8,T,140.0,181.0,37030.0,32326.0,69347.607,19:15:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307629,21-09-2021,13:02:41,27,36,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,1152,NaN,MEUDON,NaN,50.844895,4.416773,NaN,NaN,0.0,NaN,1152,workdays,46961,27,B,NaN,132.0,NaN,NaN,46952.692,13:02:32
14307630,21-09-2021,13:02:41,27,36,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,1152,NaN,MEUDON,NaN,50.844895,4.416773,NaN,NaN,0.0,NaN,1152,workdays,46961,27,B,NaN,132.0,NaN,NaN,46952.692,13:02:32
14307631,21-09-2021,13:03:13,27,132,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,1152,NaN,MEUDON,NaN,50.844895,4.416773,NaN,NaN,0.0,NaN,1152,workdays,46993,27,B,36.0,360.0,46961.0,32.0,46962.538,13:02:42
14307632,21-09-2021,00:18:14,27,58,1150,NaN,ZEPHYR,NaN,50.847132,4.421532,NaN,NaN,0.0,NaN,1150,2289,NaN,MONTGOMERY,NaN,50.837869,4.407055,NaN,NaN,0.0,NaN,2289,workdays,1094,27,B,NaN,307.0,NaN,NaN,1080.615,00:18:00
